In [1]:
import alpdesign as alp
from alpdesign.seq import *
from alpdesign.utils import *
import jax 
from jax_unirep import get_reps
import haiku as hk

In [2]:
print(forward_seqprop)

Transformed(init=<function without_state.<locals>.init_fn at 0x7f6e2062d560>, apply=<function without_state.<locals>.apply_fn at 0x7f6e2062d5f0>)


In [2]:
target_char = ['G','I','G','A','V','L','K','V','L','T','T','G','L','P','A','L','I','S','W','I','K','R','K','R','Q','Q']
oh_vec = encode_seq(target_char)
target_seq = ['GIGAVLKVLTTGLPALISWIKRKRQQ']
target_rep = get_reps(target_seq)[0]
key = jax.random.PRNGKey(0)
key, logits_key, r_key, b_key = jax.random.split(key, num=4)

batch_size = 32
bag_num = 6
logits = jax.random.normal(logits_key, shape=(jnp.shape(oh_vec)))
b_logits = jax.random.normal(logits_key, shape=(batch_size,*jnp.shape(oh_vec)))
#batch_keys = jax.random.split(key, num=batch_size)
#r = jax.random.normal(r_key)
#b = jax.random.normal(b_key)

params = forward_seqprop.init(key, logits)
#forward_seqprop = hk.transform(forward_seqprop)
beam_loss_traces, beam_loss, beam_seqs = beam_train(batch_key, target_rep, b_logits, params, train_seqprop_adam, batch_size, bag_num)

ValueError: too many values to unpack (expected 2)